In [17]:
import re
import os
import nltk
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
nltk.download() ## download packages for stop words

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
all_data = pd.read_csv('spam.csv')
all_data['label'] = all_data['text_type'].map({'ham': 1,'spam': 0})

In [ ]:
def data_preprocessing(raw_text):
    
    raw_text = raw_text.lower()
    filter_pattern = re.compile('[%s]' % re.escape(string.punctuation))
    words_only = filter_pattern.sub('', raw_text)
    raw_words = nltk.word_tokenize(words_only)
    wordnet_lematizer = WordNetLemmatizer()
    words = [wordnet_lematizer.lemmatize(raw_word) for raw_word in raw_words]
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    proc_text = ' '.join(filtered_words)
    
    return proc_text

In [ ]:
all_data['proc_text'] = all_data['text'].apply(data_preprocessing)
all_data = all_data[all_data['proc_text'] != '']
train_data, test_data = train_test_split(all_data, test_size=.2, random_state=0)

In [ ]:
train_proc_text = train_data['proc_text'].values
test_proc_text = test_data['proc_text'].values

tfidf_vectorizer = TfidfVectorizer()
train_tfidf_feat = tfidf_vectorizer.fit_transform(train_proc_text).toarray()
test_tfidf_feat = tfidf_vectorizer.transform(test_proc_text).toarray()

count_vectorizer = CountVectorizer()
train_count_feat = count_vectorizer.fit_transform(train_proc_text).toarray()
testcount_feat = count_vectorizer.transform(test_proc_text).toarray()

X_train = np.hstack((train_tfidf_feat, train_count_feat))
X_test = np.hstack((test_tfidf_feat, testcount_feat))

y_train = train_data['label'].values
y_test = test_data['label'].values

In [16]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)

print('acc score：', accuracy_score(y_test, y_pred))
print('AUC score：', roc_auc_score(y_test, y_pred))

acc： 0.8940754039497307
AUC： 0.891923436041083
